# Proceso ETL: Extracción desde Azure SQL y Carga en SQL Server Local

Este Notebook realiza un proceso ETL automático que extrae datos desde Azure SQL Database, los transforma con Pandas y los carga en SQL Server Local. Se trabajan las siguientes tablas: **Hechos**, **clientes**, **zona**, **tiempo** y **productos**.

El proceso incluye:
- Configuración de conexiones.
- Definición de consultas (almacenadas en archivos SQL).
- Especificación de claves primarias y foráneas.
- Creación dinámica de las tablas en SQL Server Local.
- Carga de datos.


In [8]:
import pyodbc         # Conexión con bases de datos SQL Server
import pandas as pd   # Manipulación y transformación de datos
import numpy as np    # Gestión de datos numéricos
import os             # Operaciones con archivos y rutas
import warnings       # Control de advertencias

# Suprimir avisos innecesarios
warnings.filterwarnings("ignore", category=UserWarning)

## CONFIGURACIÓN DE CONEXIONES

In [9]:
# Conexión a Azure SQL Database
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'
azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

# Conexión a SQL Server Local
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

## DEFINICIÓN DE CONSULTAS Y METADATOS

In [10]:
# Ubicación de los archivos SQL que contienen las consultas de extracción
query_folder = "Modelo_dimensional"
queries = {
    "hechos": "hechos.sql",
    "clientes": "clientes.sql",
    "zona": "zona.sql",
    "tiempo": "tiempo.sql",
    "productos": "productos.sql"
}

# Claves primarias para cada tabla
primary_keys = {
    "hechos": ["CODE"],
    "clientes": ["Customer_ID"],
    "zona": ["TIENDA_ID"],
    "tiempo": ["Fecha"],
    "productos": ["Id_Producto"]
}

# Relaciones (claves foráneas) definidas para la tabla de hechos
foreign_keys = {
    "hechos": {
        "Customer_ID": "clientes(Customer_ID)",
        "TIENDA_ID": "zona(TIENDA_ID)",
        "Id_Producto": "productos(Id_Producto)",
        "Sales_Date": "tiempo(Fecha)"
    }
}

## FUNCIÓN PARA CREAR TABLAS EN SQL SERVER LOCAL

In [11]:
def create_table_sql(table_name, df):
    # Especificación manual para columnas de fecha según cada tabla
    date_columns = {
        "clientes": ["Fecha_nacimiento"],
        "tiempo": ["InicioMes", "FinMes", "Fecha"],
        "hechos": ["DATE_ULTIMA_REVISION", "Logistic_date", "Prod_date", "Sales_Date"]
    }
    
    col_defs = []
    for col in df.columns:
        # Si la columna figura en el mapeo manual de fechas
        if table_name in date_columns and col in date_columns[table_name]:
            col_defs.append(f'[{col}] DATE')
        elif np.issubdtype(df[col].dtype, np.datetime64):
            col_defs.append(f'[{col}] DATE')
        elif df[col].dtype == np.float32:
            col_defs.append(f'[{col}] FLOAT')
        elif df[col].dtype == np.int32:
            col_defs.append(f'[{col}] INT')
        else:
            # Para textos, calcular el tamaño óptimo con un margen (máximo 2000)
            max_len = df[col].astype(str).str.len().max()
            varchar_size = min(2000, max(1, int(max_len * 1.3)))
            col_defs.append(f'[{col}] NVARCHAR({varchar_size})')
    
    # Incorporar clave primaria si se ha definido
    pk = ", PRIMARY KEY (" + ", ".join(primary_keys[table_name]) + ")" if table_name in primary_keys else ""
    
    # Agregar las relaciones de claves foráneas, si existen
    fk = ""
    if table_name in foreign_keys:
        for col, ref in foreign_keys[table_name].items():
            fk += f", FOREIGN KEY ({col}) REFERENCES {ref}"
    
    return f"CREATE TABLE {table_name} ({', '.join(col_defs)}{pk}{fk});"


# -------------------------
# FUNCIÓN PARA ELIMINAR TABLAS EN ORDEN (para respetar dependencias)
# -------------------------
def drop_tables_in_order(cursor, conn):
    # Orden inverso para eliminar primero las tablas dependientes
    drop_order = ["hechos", "tiempo", "productos", "zona", "clientes"]
    for table in drop_order:
        query = f"IF OBJECT_ID('{table}', 'U') IS NOT NULL DROP TABLE {table};"
        try:
            cursor.execute(query)
            conn.commit()
        except Exception as e:
            print(f"Error al borrar la tabla {table}: {e}")

## EJECUCIÓN DEL PROCESO ETL

In [12]:
try:
    # Conexión a Azure SQL y a SQL Server Local
    conn_azure = pyodbc.connect(azure_conn_str)
    conn_local = pyodbc.connect(local_conn_str)
    print("Conexiones establecidas exitosamente.")
    
    # Eliminar tablas existentes en el orden adecuado
    with conn_local.cursor() as cursor:
        drop_tables_in_order(cursor, conn_local)
        print("Tablas existentes eliminadas correctamente.")
    
    # Procesar cada tabla definida en el diccionario de consultas
    for table_name, file_name in queries.items():
        print(f"\nExtrayendo datos para la tabla '{table_name}'...")
        query_path = os.path.join(query_folder, file_name)
        
        with open(query_path, "r", encoding="utf-8") as file:
            sql_query = file.read()
        
        # Extraer datos desde Azure SQL
        df = pd.read_sql(sql_query, conn_azure)
        if df.empty:
            print(f"La consulta para '{table_name}' no devolvió resultados. Se omite.")
            continue
        
        # Reemplazar valores nulos en columnas numéricas y ajustar tipos
        df = df.fillna(0)
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)
        
        # Crear la tabla en SQL Server Local con la estructura detectada
        with conn_local.cursor() as cursor:
            create_sql = create_table_sql(table_name, df)
            cursor.execute(create_sql)
            conn_local.commit()
            print(f"Tabla '{table_name}' creada en SQL Server Local.")
            
            # Insertar los datos extraídos
            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()
            print(f"{df.shape[0]} registros insertados en '{table_name}'.")
    
except Exception as e:
    print(f"Error durante el proceso ETL: {e}")
    
finally:
    conn_azure.close()
    conn_local.close()

print("\nProceso ETL completado exitosamente.")

Conexiones establecidas exitosamente.
Tablas existentes eliminadas correctamente.

Extrayendo datos para la tabla 'hechos'...
Error durante el proceso ETL: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]El nombre de columna 'CODE' no existe en la vista o tabla de destino. (1911) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]No se pudo crear la restricción o el índice. Vea los errores anteriores. (1750)")

Proceso ETL completado exitosamente.


### Resumen del Proceso ETL

- **Extracción:** Se obtienen datos de Azure SQL Database mediante consultas definidas en archivos SQL (para las tablas: hechos, clientes, zona, tiempo y productos).
- **Transformación:** Los datos se manipulan en Pandas, se convierten tipos de datos, se gestionan valores nulos y se prepara la estructura de las tablas.
- **Carga:** Se crean las tablas en SQL Server Local respetando claves primarias y foráneas y se insertan los registros.

Este Notebook automatiza el proceso ETL y permite monitorizar cada paso a través de los mensajes en consola.